<a href="https://colab.research.google.com/github/eisbetterthanpi/vision/blob/main/selenium_lens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://lens.google.com/search?p=0


In [ ]:
# @title setup
# https://colab.research.google.com/github/kaliiiiiiiiii/Selenium-Profiles/blob/master/google-colab/selenium_profiles.ipynb

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium

# Install xvfb
apt install -y xvfb

# Install Selenium-Profiles
pip uninstall -y selenium_profiles
pip install --no-cache-dir selenium_profiles>=2.2.6

# pip install https://github.com/kaliiiiiiiiii/Selenium-Profiles/archive/refs/heads/dev.zip # dev-branch

# install python packages
pip install google-colab-shell
pip install webdriver-manager
pip install Pyvirtualdisplay


In [ ]:
# @title service
# https://stackoverflow.com/questions/76428999/selenium-chrome-webdriver-stopped-working
# https://stackoverflow.com/questions/76409097/driver-webdriver-chrome-issues-with-a-selenium-approach-how-to-work-aro

!apt-get update
!apt-get install chromium chromium-driver
!pip3 install selenium

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

service = Service(executable_path="chromedriver")
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
# options.add_argument('--blink-settings=imagesEnabled=false') # me
# options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
# driver = webdriver.Chrome(service=service, options=options)

# url = "https://www.google.com/search?q=Semi-Detached+House+bungalow+modern+-plan+-interior+-illustration+-news+-model+-3D"
# driver.get(url)  # test fingerprint
# print(driver.title)
# driver.quit()


In [3]:
# @title scrape google
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import requests
import math
from PIL import Image
import matplotlib.pyplot as plt

url = "https://lens.google.com/search?p=0"


# service = Service(executable_path="chromedriver")
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# # options.add_argument('--blink-settings=imagesEnabled=false') # me

# driver = webdriver.Chrome(service=service, options=options)

# driver.get(url)

# reject = driver.find_elements(By.XPATH,"*//button[@aria-label='Reject all']")#[0]#.click()
# if reject: reject[0].click()

# hits = driver.find_elements(By.XPATH,"*//a[@class='wXeWr islib nfEiy']")
# print(len(hits))
# # print(len(img_urls))


In [4]:
# @title funcs

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
def newdriver():
    service = Service(executable_path="chromedriver")
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--blink-settings=imagesEnabled=false') # me dont load images
    # options.add_argument("--disable-gpu") # me against SessionNotCreatedException: Timed out receiving message from renderer
    driver = webdriver.Chrome(service=service, options=options)
    return driver

import signal
import time
def timeout_handler(num, stack): raise Exception("timeout")

def new_driver():
    for i in range(30):
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(2)
        try: return newdriver()
        except Exception as ex: pass
        finally: signal.alarm(0)


# driver = new_driver()
# driver.get(url)
# reject = driver.find_elements(By.XPATH,"*//button[@aria-label='Reject all']")#[0]#.click()
# if reject: reject[0].click()
# # # driver.quit()

def scroll_end(driver):
    end = driver.find_elements(By.XPATH,'''*//div[@data-endedmessage="Looks like you've reached the end"]''')[0]#.click()
    while end.get_attribute("data-status") != "3": # =5, =3 when reach end
        more = driver.find_elements(By.XPATH,"*//input[@value='Show more results']")[0]#.click()
        if more.is_displayed(): more.click()
        # height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # scroll down



# getting img url by clicking in, not reliable
#     # try: img_src = driver.find_elements(By.XPATH,"*//img[@jsname='kn3ccd']")[0].get_attribute("src")
#     # except: continue
#     # img_src = driver.find_elements(By.XPATH,"*//a[@class='Du2c7e']")[0].get_attribute("href")
#     # img_src = driver.find_elements(By.XPATH,"*//a[@class='Du2c7e']/img")[0].get_attribute("src")

def click_all(driver):
    see_more=[]
    hits = driver.find_elements(By.XPATH,"*//a[@class='wXeWr islib nfEiy']")
    driver.execute_script("hits = document.getElementsByClassName('wXeWr islib nfEiy')")
    print(len(hits)) # 780
    # for i, x in enumerate(hits):
    for i in range(len(hits)):
        # hits[i].click()
        driver.execute_script("hits["+str(i)+"].click();")
        try:
            WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "*//a[@aria-label='See more Related content']")))
            see_mor = driver.find_elements(By.XPATH,"*//a[@aria-label='See more Related content']")[0].get_attribute("href")
        except: see_mor="_"
        see_more.append(see_mor)
    return see_more

from urllib.parse import unquote
import re
def get_imgurls(driver):
    pro=driver.page_source
    # pro=driver.execute_script("return document.documentElement.outerHTML")
    pro = unquote(pro)
    pro = pro.encode().decode('unicode-escape') # try to remove \\u00
    # pattern = r"https://(?:(?!\").)*\.jpg" # start with https:// , no " , end with .jpg # https://stackoverflow.com/a/67540659/13359815
    pattern = r"(?<=href=\"/imgres\?imgurl=)(?:(?!&amp;).)*(?=&amp;)" # start with href="/imgres?imgurl= , no &amp; , end with &amp; # https://stackoverflow.com/a/3926546/13359815
    # pattern = r"https://(?:(?!\").)*(?=\")" # start with https:// , no " , end with " # all urls
    # pattern = r"(?<=\],\[\")https://(?:(?!\").)*(?=\")" # start with ],[" then https:// , no " , end with " # all imgs? fast
    m = re.findall(pattern, pro)
    return m

import time
def get_skip(id=0, tt=20, dx=20):
    now = time.time()
    cyc = tt*dx
    curr = now %(cyc)
    want = id*dx
    skp = (want-curr)%cyc
    return skp



In [ ]:
# @title works
driver = new_driver()

url = "https://lens.google.com/search?p=0"
driver.get(url)

# upload
WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "*//button[@jscontroller='soHxf']")))
element = driver.find_elements(By.XPATH,"*//button[@jscontroller='soHxf']")[0]
element.click()

# by computer
WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "*//li[@aria-label='Computer']")))
element = driver.find_elements(By.XPATH,"*//li[@aria-label='Computer']")[0]
element.click()

# input appears
WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "*//input[@type='file']")))
element = driver.find_elements(By.XPATH,"*//input[@type='file']")[0]

file = "/content/01. 1 LUCKY RISE 2022-09.jpg"
element.send_keys(file)



In [ ]:
# @title check driver state
from PIL import Image
import matplotlib.pyplot as plt
driver.get_screenshot_as_file("canvas.png")
image = Image.open('canvas.png').convert("RGB")
plt.figure(figsize=(5, 3))
plt.axis('off')
plt.imshow(image)
plt.show()

# print(driver.page_source)


In [ ]:
# @title get lens urls
from urllib.parse import unquote
import re

pro=driver.page_source
# pro=driver.execute_script("return document.documentElement.outerHTML")
pro = unquote(pro)
pro = pro.encode().decode('unicode-escape') # try to remove \\u00
pattern = r"(?<=src\=\")https://encrypted-tbn[0-9].gstatic.com/images(?:(?!\").)*(?=\")" # start with src=" ,then https...tbn(any num).../images , no " , end with ""
m = re.findall(pattern, pro)

# print(m)
print(len(m)) # 60


In [5]:
# @title works funcs
# driver = new_driver()

# try: driver.get(url)
# except Exception as ex:
#     print(url, ex)

def works(driver, file):
    url = "https://lens.google.com/search?p=0"
    driver.get(url)
    reject = driver.find_elements(By.XPATH,"*//button[@aria-label='Reject all']")#[0]#.click()
    if reject: reject[0].click()
    # upload
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "*//button[@jscontroller='soHxf']")))
    element = driver.find_elements(By.XPATH,"*//button[@jscontroller='soHxf']")[0]
    element.click()
    # by computer
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "*//li[@aria-label='Computer']")))
    element = driver.find_elements(By.XPATH,"*//li[@aria-label='Computer']")[0]
    element.click()
    # input appears
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "*//input[@type='file']")))
    element = driver.find_elements(By.XPATH,"*//input[@type='file']")[0]
    element.send_keys(file)

from urllib.parse import unquote
import re
def get_lensurls(driver):
    pro=driver.page_source
    # pro=driver.execute_script("return document.documentElement.outerHTML")
    pro = unquote(pro)
    pro = pro.encode().decode('unicode-escape') # try to remove \\u00
    pattern = r"(?<=src\=\")https://encrypted-tbn[0-9].gstatic.com/images(?:(?!\").)*(?=\")" # start with src=" ,then https...tbn(any num).../images , no " , end with ""
    m = re.findall(pattern, pro)
    return m


# https://www.google.com/searchbyimage?image_url=<your_url>&client=app
# uuu = "https://www.google.com/searchbyimage?image_url="+m[23]+"&client=app"
def lens2google(uuu, driver):
    url = "https://www.google.com/searchbyimage?image_url="+uuu+"&client=app"
    print("lens2google", url)
    driver.get(url)
    # visually similar images
    try: WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "*//a[@class='ekf0x hSQtef']")))
    except: return
    href = driver.find_elements(By.XPATH,"*//a[@class='ekf0x hSQtef']")[0].get_attribute("href")
    # print(href)
    hh = href.split('&')[1]
    # print(hh)
    search = 'house'
    exclude = " -youtube -plan -interior -illustration -news -model -3D -aerial"
    query = (search+exclude).replace(' ','+')
    # print('https://www.google.com/search?client=app&'+hh+'&q='+query+'&tbm=isch')
    return 'https://www.google.com/search?client=app&'+hh+'&q='+query+'&tbm=isch'




In [ ]:

driver = new_driver()
filename = "/content/01. 1 LUCKY RISE 2022-09.jpg"
works(driver, filename)
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "*//img[@class='wETe9b jFVN1']")))
m = get_lensurls(driver)
print(len(m))


60


<ipython-input-20-6f7abe12c04d>:26: DeprecationWarning: invalid escape sequence '\s'
  pro = pro.encode().decode('unicode-escape') # try to remove \\u00


In [ ]:
# !unzip /content/drive/MyDrive/select.zip -d /
!unzip /content/drive/MyDrive/select.zip -d /content


In [ ]:
import os
from PIL import Image

driver = new_driver()

t=0
urllst=[]
for x in range(1,7):
    img_dir = '/content/select/0'+str(x)
    for filename in os.listdir(img_dir):
    # filelist = os.listdir(img_dir):
        # print(filename)
        file = img_dir+'/'+filename
        print(file)
        works(driver, file)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "*//img[@class='wETe9b jFVN1']")))
        m = get_lensurls(driver)

        # # lens2google(uuu, driver)
        # e = ThreadPoolExecutor(len(filelist))
        # future = [e.submit(lens2google, uuu, driver) for uuu in m]
        # result = [f.result() for f in future if f.result() is not None]
        for uuu in m:
            urllst.append(lens2google(uuu, driver))


# # save to final
data = {'img_url':urllst}
df = pd.DataFrame(data)
# display(df)
with pd.ExcelWriter('/content/drive/MyDrive/lens_urls.xlsx', mode='a', engine="openpyxl", if_sheet_exists="overlay") as writer:
    startrow = len(list(list(writer.sheets.values())[0].rows))
    df.to_excel(writer, header=None, index=None, startrow=startrow)

# # add to scrape
# data = {'search_url':[small[i] for i in good+great]}
# df = pd.DataFrame(data)
# # display(df)
# with pd.ExcelWriter('/content/drive/MyDrive/toscrape.xlsx', mode='a', engine="openpyxl", if_sheet_exists="overlay") as writer:
#     startrow = len(list(list(writer.sheets.values())[0].rows))
#     df.to_excel(writer, header=None, index=None, startrow=startrow)


In [ ]:
# @title wwwwwwwww
import pandas as pd

def to_save(df, id=0, tt=2, dx=10):
    print(id,'done')
    with pd.ExcelWriter('/content/drive/MyDrive/lens_urls.xlsx', mode='a', engine="openpyxl", if_sheet_exists="overlay") as writer:
        startrow = len(list(list(writer.sheets.values())[0].rows))
        df.to_excel(writer, header=None, index=None, startrow=startrow)


# def get_search(sid=0, stt=5, sdx=30, num=5):
#     if num <= 0: return
#     df_scrape = pd.read_excel("/content/drive/MyDrive/toscrape.xlsx", engine='openpyxl') # search urls to scrape
#     done_idx = df_scrape['done'].count()
#     print("done_idx",done_idx)
#     # df_scrape.loc[done_idx, 'done'] = 1
#     df_scrape.loc[done_idx:done_idx+num, 'done'] = 1
#     df_scrape.to_excel('/content/drive/MyDrive/toscrape.xlsx', index=False)
#     searchurls = df_scrape.loc[done_idx:done_idx+num, 'search_url'].tolist()
#     return searchurls


# def nin(): # time is right
#     global uq
#     if uq: return
#     uq=True
#     skip = get_skip(id=id, tt=5, dx=30)
#     time.sleep(skip)
#     global urlq
#     searchurls = get_search(sid=id, num=5-len(urlq))
#     if searchurls: urlq.extend(searchurls)
#     uq=False


def pon(): # time is right
    global sq
    if sq: return
    sq=True
    skip = get_skip(id=id, tt=2, dx=10)
    time.sleep(skip)
    global ulist
    data = {'url':ulist}
    df = pd.DataFrame(data)
    to_save(df, id=id)
    ulist = []
    sq=False


from concurrent.futures import ThreadPoolExecutor

driver = new_driver()
e = ThreadPoolExecutor(30)

id=0 # 0:0-90, 1:90-175

sq=False
# uq=False
urlq = []
ulist = []
# while True:
    # ufuture = e.submit(nin)
    # if len(urlq)==0: ufuture.result() # need to wait
    # url = urlq.pop(0)
    # print(url)

import os
# for x in range(1,7):
# img_dir = '/content/select/0'+str(x)
img_dir = '/content/select/02' # 1M 2B 3S 4A 5t
for i, filename in enumerate(os.listdir(img_dir)[:90]):
# filelist = os.listdir(img_dir):
    # print(filename)
    file = img_dir+'/'+filename
    print(i, file)
    works(driver, file)
    try: WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "*//img[@class='wETe9b jFVN1']")))
    except Exception as ex:
        print(ex)
        continue
    m = get_lensurls(driver)

    for uuu in m:
        u = lens2google(uuu, driver)
        if u: ulist.append(u)

    ulist.extend(u)
    sfuture = e.submit(pon)


# e = ThreadPoolExecutor(len(filelist))
# future = [e.submit(lens2google, uuu, driver) for uuu in m]
# result = [f.result() for f in future if f.result() is not None]



/content/select/02/02. 116 FARRER RD 2022-05.jpg
Message: 
Stacktrace:
#0 0x592c8a5e47f9 <unknown>
#1 0x592c8a5843b3 <unknown>
#2 0x592c8a2cc016 <unknown>
#3 0x592c8a30081e <unknown>
#4 0x592c8a3368fb <unknown>
#5 0x592c8a323ded <unknown>
#6 0x592c8a3349e1 <unknown>
#7 0x592c8a323c93 <unknown>
#8 0x592c8a2f5ce4 <unknown>
#9 0x592c8a2f74d2 <unknown>
#10 0x592c8a5b0542 <unknown>
#11 0x592c8a5bfce7 <unknown>
#12 0x592c8a5bf9e4 <unknown>
#13 0x592c8a5c413a <unknown>
#14 0x592c8a5c05b9 <unknown>
#15 0x592c8a5a5e00 <unknown>
#16 0x592c8a5d75d2 <unknown>
#17 0x592c8a5d7778 <unknown>
#18 0x592c8a5efa1f <unknown>
#19 0x7b1d83df7609 start_thread
#20 0x7b1d8300a133 clone
 /content/select/02/02. 116 FARRER RD 2022-05.jpg


<ipython-input-5-711dfbf50934>:32: DeprecationWarning: invalid escape sequence '\s'
  pro = pro.encode().decode('unicode-escape') # try to remove \\u00


lens2google https://www.google.com/searchbyimage?image_url=https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQNoonlZ5Gtl3pIy2gY1uM8uYdqPSBnZFpl41WIqcY0npBN_LL3&client=app
lens2google https://www.google.com/searchbyimage?image_url=https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcQ9xBWFyV0UN_hR7DydlIZfamAptFo1Tu-8eyFY7wBGh7t14_iHp6vJKS05k-7M82UhNGc&client=app
lens2google https://www.google.com/searchbyimage?image_url=https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcS5CH4K6TfolOn1pTJmrYu5L6jWO73CUg_m-5we7QI5ahbZrerfaF0baGEwJMGCJNwRwHc&client=app
lens2google https://www.google.com/searchbyimage?image_url=https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQNoonlZ5Gtl3pIy2gY1uM8uYdqPSBnZFpl41WIqcY0npBN_LL3&client=app
lens2google https://www.google.com/searchbyimage?image_url=https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcQFX95VTgt70Ano_dB8EiXtJ5DJXuPfQ2hhF3BdXt4JIj4b9Of-&client=app
lens2google https://www.google.com/searchbyimage?image_url=https://encrypted-tbn2.gstat

<ipython-input-5-711dfbf50934>:32: DeprecationWarning: invalid escape sequence '\s'
  pro = pro.encode().decode('unicode-escape') # try to remove \\u00


lens2google https://www.google.com/searchbyimage?image_url=https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcQoEzfHR6MN90mGAPKfMV-fC0oTZrJUXoOAsh2gTZ3phifCNxPg&client=app
lens2google https://www.google.com/searchbyimage?image_url=https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcQHEtQAVY07nRpYkS49zSmweFC0qjb2xIJkpu6S49dACebytdek&client=app
lens2google https://www.google.com/searchbyimage?image_url=https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcT_8-t501HP5W3eQjM8zVcBDkgrP2P8ww7bqZnitRgpu4CrOQNw&client=app
lens2google https://www.google.com/searchbyimage?image_url=https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcRXcvVMzRrAgdru9qp0DznoxCmD1w2_mMLtHtDIayF2xlQuwXy7&client=app
lens2google https://www.google.com/searchbyimage?image_url=https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcSKn8YCRy9s_hG4PRJ-blEXKg8unGqTt9uZSrsDCpTPQrd_VwSF&client=app
lens2google https://www.google.com/searchbyimage?image_url=https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQy1kAVvvu13M